In [207]:
# using Pkg
# Pkg.add(url="https://github.com/MasanoriKanamaru/Astroshaper")
# Pkg.update("Astroshaper")
# Pkg.test("Astroshaper")

In [194]:
# using Plots
# using FileIO
# using JLD2
# using Profile

In [2]:
using Revise
using Astroshaper

In [3]:
shapedir = "/Users/masanorikanamaru/Documents/shape/ryugu"

# shapename = "ryugu_v252_f500.obj"
# shapename = "ryugu_v752_f1500.obj"
shapename = "ryugu_test.obj"

# shapename = "SHAPE_SPC_49k_v20190802.obj"
# shapename = "SHAPE_SPC_200k_v20190802.obj"
# shapename = "SHAPE_SPC_800k_v20190802.obj"
# shapename = "SHAPE_SPC_3M_v20190802.obj"

# shapename = "SHAPE_SFM_49k_v20180804.obj"
# shapename = "SHAPE_SPC_49k_v20200323.obj"
# shapename = "SHAPE_SPC_3M_v20200323.obj"

shapepath = joinpath(shapedir, shapename)
@show shapepath;

shapepath = "/Users/masanorikanamaru/Documents/shape/ryugu/ryugu_test.obj"


In [4]:
# shapepath = splitext(shapepath)[1] * ".jld2"

In [13]:
shape = Shape(shapepath; scale=1000, find_visible_facets=true, save_shape=false)

Shape model
-----------
Nodes             : 2976
Faces             : 5932
Surface area      : 2.6974896578683443e6
Volume            : 3.768813053271797e8
Equivalent radius : 448.0969577371473
Center-of-Figure  : [0.37905838040179374, 0.23475514243730786, -0.03365988225760159]
Inertia tensor    : 
    | Ixx Ixy Ixz |   [0.0, 0.0, 0.0]
    | Iyx Iyy Iyz | = [0.0, 0.0, 0.0]
    | Izx Izy Izz |   [0.0, 0.0, 0.0]


In [15]:
# draw(shape)
# draw(shape, data=:radius)
# draw(shape, data=:temperature)
# draw(shape; data=:illumination, r̂☉=[1,0,0.])
# draw(shape, data=[center[3] for center in shape.facets.center])

In [16]:
orbit = OrbitalElements(RYUGU)

Orbital elements
----------------


Semi-mojor axis        : a  = 1.18956373 [AU]
Eccentricity           : e  = 0.19027921 [-]
Lon. of ascending node : Ω  = 251.589203 [deg]
Argument of periapsis  : ω  = 211.435963 [deg]
Inclination            : I  = 5.8840222 [deg]
Periapsis passage time : tₚ = -2.4948038004043424e6 [sec]
Mean anomaly           : Φ  = 21.9353799 [deg]

Other parameters
----------------
Gravitational parameter : μ = 1.32712440018e20 [m^3/s^2]
Mean motion             : n = 0.7596656791419168 [deg/day]
Orbital period          : T = 473.8926739544681 [day]


In [17]:
spin = SpinParams(RYUGU, orbit)

Spin parameters
---------------


Right ascension (RA) : α = 96.4 [deg]
Declination (Dec)    : δ = -66.4 [deg]
Ecliptic longitude   : λ = 179.33757188938182 [deg]
Ecliptic latitude    : β = -87.44207056697601 [deg]
Obliquity            : ε = 171.65664649001607 [deg]
Spin period          : P = 7.63262 [hours]
Spin rate            : ω = 0.00022866712242903872 [rad/sec]
Vernal equinox lon.  : γ = 154.3323208882217 [deg]
                           (longitude from the periheion direction)


In [8]:
# Astroshaper.init_temps_zero!(shape, orbit, spin, params_thermo)
# Astroshaper.init_temps_zero!(shape::Shape, collect(1:shape.num_face))

# YORP効果 @現在の軌道

In [49]:
thermo_params = ThermoParams(
    A_B   = 0.04,
    # A_B   = fill(0.04, shape.num_face),
    A_TH  = 0.0,
    k     = 0.1,
    # k     = fill(0.1, shape.num_face),
    ρ     = 1270.0,
    Cp    = 600.0,
    ϵ     = 1.,
    t_bgn = 0.0,
    t_end = spin.P * 10,
    Nt    = 721,
    z_max = 0.6,
    Nz    = 41,
    P     = spin.P,
);

In [50]:
typeof(thermo_params)

ThermoParams{Int64, Float64, Int64, Float64}

In [52]:
@time τ̄ = run_YORP(shape, orbit, spin, thermo_params)

  7.546398 seconds (2 allocations: 160 bytes)


3-element Vector{Float64}:
  2.5306956279461983
 -8.233792225809143
 -0.7075406040546186

In [53]:
C = 4.039541372643629e16
ω̇, ωε̇, ωψ̇ = torque2rate(τ̄, spin, C)  # [deg/day²]

@show shapename
@show τ̄
@show ω̇, ωε̇, ωψ̇
@show YORP_timescale(3.5, 7.63262, ω̇);  # 3.5時間から7.6時間まで減速する時間スケール [yr]

shapename = "ryugu_test.obj"
τ̄ = [2.5306956279461983, -8.233792225809143, -0.7075406040546186]
(ω̇, ωε̇, ωψ̇) = (-2.305600098551757e-6, 6.734927363385765e-5, 6.191331835096895e-5)
YORP_timescale(3.5, 7.63262, ω̇) = 1.5871695526790055e6


In [15]:
##############################
#  　ryugu_v252_f500.obj     #
#　  t_end = orbit.T         #
##############################

# shapename = "ryugu_v252_f500.obj"
# τ̄ = [2.102139762963553, -1.9503690549955461, 0.10649756611585151]
# (ω̇, ωε̇, ωψ̇) = (-2.417553631012555e-6, 8.713420516412938e-6, 2.9006181145065934e-5)
# getTimeScale(3.5, 7.63262, ω̇) = 1.514706707465591

##############################
#　  ryugu_test.obj          #
#　  t_end = spin.P * 10     #
##############################

# shapename = "ryugu_test.obj"
# τ̄ = [2.5306956279461983, -8.233792225809143, -0.7075406040546186]
# (ω̇, ωε̇, ωψ̇) = (-2.305600098551757e-6, 6.734927363385765e-5, 6.191331835096895e-5)
# YORP_timescale(3.5, 7.63262, ω̇) = 1.5871695526790055e6

# Yarkovsky効果 @現在の軌道

可視光の反射率を変えた時に、Yarkovksy drift rateがどれくらい変わるか

- 全球を白く塗る
- 全球を黒く塗る
- 半球を白く塗る 
- 半球を黒く塗る
- ある場所から半径R圏内を白く塗る
- ある場所から半径R圏内を黒く塗る

In [56]:
thermo_params = ThermoParams(
    A_B   = 0.04,
    # A_B   = fill(0.04, shape.num_face),
    A_TH  = 0.0,
    k     = 0.1,
    # k     = fill(0.1, shape.num_face),
    ρ     = 1270.0,
    Cp    = 600.0,
    ϵ     = 1.,
    t_bgn = 0.0,
    t_end = spin.P * 10,
    Nt    = 721,
    z_max = 0.6,
    Nz    = 41,
    P     = spin.P,
);

In [47]:
A_B = fill(0.04, shape.num_face)

selected = [center[1] > 0 for center in shape.facets.center]
A_B[selected] .= 1;

In [48]:
draw(shape; data=A_B)

GLMakie.Screen(...)

In [128]:
@time df = run_TPM(shape, orbit, spin, thermo_params)

  7.418221 seconds (8.44 k allocations: 239.047 KiB)


,t,u,ϕ,f_x,f_y,f_z,τ_x,τ_y,τ_z
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.0,0.468819,0.0,1.90212e-5,1.02542e-5,-1.40544e-6,-0.000528987,0.000193509,-9.0529e-5
2,381.631,0.46889,0.0872665,0.025544,0.0134518,-0.00159032,0.43984,-0.671912,-0.214646
3,763.262,0.46896,0.174533,0.0517712,0.0262483,-0.00324469,0.838556,-1.27199,-0.415826
4,1144.89,0.469031,0.261799,0.079729,0.0384986,-0.00504891,1.21761,-1.83762,-0.591526
5,1526.52,0.469101,0.349066,0.107593,0.049073,-0.00683545,1.56989,-2.37126,-0.749464
6,1908.15,0.469172,0.436332,0.134406,0.0575789,-0.00859515,1.90292,-2.85768,-0.869135
7,2289.79,0.469242,0.523599,0.159602,0.0639084,-0.010362,2.21003,-3.30579,-0.955768
8,2671.42,0.469313,0.610865,0.182878,0.0679558,-0.0119916,2.48997,-3.75187,-1.05517
9,3053.05,0.469383,0.698132,0.203958,0.0700833,-0.0136593,2.73357,-4.18318,-1.16108


In [28]:
# draw(shape; data=[flux.sun for flux in shape.facets.flux])
# draw(shape; data=[flux.rad for flux in shape.facets.flux])

In [23]:
using Distributed

In [52]:
# nworkers()
# nprocs()
# addprocs(4)
# rmprocs(2)

In [53]:
function f1()
    nheads = @distributed (+) for i = 1:200000000
        Int(rand(Bool))
    end
end

function f2()
    sum(Int(rand(Bool)) for i = 1:200000000)
end

f2 (generic function with 1 method)

In [54]:
@time f1()

  0.133683 seconds (44.48 k allocations: 2.289 MiB, 27.05% compilation time)


99998616

In [55]:
@time f2()

  0.243791 seconds


100010017

In [ ]:
# open("ryugu_test_temps.txt","w") do f
#     for smesh in shape.smeshes
#         println(f, smesh.Tz[begin], ", ", smesh.flux.sun, ", ", smesh.flux.scat, ", ", smesh.flux.rad)
#     end
# end

In [ ]:
# shapename = "ryugu_v252_f500.obj"

# Thermophysical parameters
# -------------------------
# A_B   : 0.04
# A_TH  : 0.0
# k     : 0.1
# ρ     : 1270.0
# Cₚ    : 600.0
# ϵ     : 1.0
# P     : 27477.432
# l     : 0.21287051812296282
# Γ     : 276.04347483684523
# Δt    : 0.01388888888888889
# t_bgn : 0.0
# t_end : 1490.1074827395094
# Nt    : 107288
# Δz    : 0.09395382778392625
# z_max : 2.8186148335177874
# Nz    : 31
# λ     : 0.12520702099737538

# 影 + 熱伝導
# τ̄ = [2.166292988343463, -1.8232052646633534, 0.12149914462601369]
# (ω̇, ωε̇, ωψ̇) = (-2.3559241586480615e-6, 7.198547956478111e-6, 2.903520943282301e-5)
# getTimeScale(3.5, 7.63262, ω̇) = 1.5543304681988876

# 影 + 熱伝導 + 放射の再吸収
# τ̄ = [2.1867628755835007, -1.9130691894522611, 0.20980627031427984]
# (ω̇, ωε̇, ωψ̇) = (-3.3918521233894786e-6, 7.818508184547549e-6, 2.964269717748932e-5)
# getTimeScale(3.5, 7.63262, ω̇) = 1.0796121314667426

# 影 + 熱伝導 + 放射の再吸収 + 散乱光による自己加熱
# τ̄ = [2.1865019814933127, -1.9132227674055484, 0.20659523329863488]
# (ω̇, ωε̇, ωψ̇) = (-3.358599466983103e-6, 7.826075560400724e-6, 2.9640911737278277e-5)
# getTimeScale(3.5, 7.63262, ω̇) = 1.0903011021560833

# 影 + 熱伝導 + 放射の再吸収 + 散乱光による自己加熱 + 熱放射の再吸収による自己加熱
# τ̄ = [2.1769952611580217, -1.9075575309460082, 0.11690136500927037]
# (ω̇, ωε̇, ωψ̇) = (-2.4174445056821524e-6, 7.953525830274134e-6, 2.9524204264352846e-5)
# getTimeScale(3.5, 7.63262, ω̇) = 1.5147750825077182

In [ ]:
# shapename = "ryugu_v752_f1500.obj"

# 影
# τ̄ = [3.3097974041391374, -1.7111464770712586, 0.12728322411992796]
# (ω̇, ωε̇, ωψ̇) = (-1.5003553780529698e-6, 9.426867138482173e-7, 3.9434066533743185e-5)
# getTimeScale(3.5, 7.63262, ω̇) = 2.440678224718057

# 影 + 熱伝導 + 放射の再吸収 + 散乱光による自己加熱 + 熱放射の再吸収による自己加熱
# τ̄ = [2.165370019439715, -1.8680106417353224, 0.027423863292429056]
# (ω̇, ωε̇, ωψ̇) = (-1.433047521048842e-6, 7.77033687219021e-6, 2.923188963996669e-5)
# getTimeScale(3.5, 7.63262, ω̇) = 2.5553128188466436

In [ ]:
# shapename = "ryugu_test.obj"

# 影
# τ̄ = [3.2623680861875415, -1.7632177354631766, 0.2112963317251145]
# (ω̇, ωε̇, ωψ̇) = (-2.484150619384357e-6, 1.5205017761112165e-6, 3.9220246601817255e-5)
# getTimeScale(3.5, 7.63262, ω̇) = 1.4740993045985396

# 影 + 熱伝導 + 放射の再吸収 + 散乱光による自己加熱 + 熱放射の再吸収による自己加熱
# τ̄ = [2.1336218786667724, -1.878807112180092, 0.04045730505339008]
# (ω̇, ωε̇, ωψ̇) = (-1.6056650245345357e-6, 7.99631943219241e-6, 2.8978423396048164e-5)
# getTimeScale(3.5, 7.63262, ω̇) = 2.280603142373392

In [ ]:
surf_temp = [T[begin] for T in Ts];

In [ ]:
plot(framestyle=:box, legend=false, size=(600,400))
plot!(xlims=(0, t_max/P))
plot!(ylims=(0, 400))

ts = collect(0:Δτ:t_max/P)
zs = collect(0:Δz:x_max/l)

plot!(ts, surf_temp)

xlabel!("Time / Rotation period")
ylabel!("Surface temperature [K]")

# View factorの計算時間

In [ ]:
plot(framestyle=:box, legend=false)


Ns = [500,      1500,     5932,     49152,      196608]    # 786432,     3145728
ts = [0.002659, 0.027669, 0.473278, 164.478600, 10829.774176]    # 3600*24*10, 3600*24*1000
plot!(Ns, ts, marker=(:circle))

plot!(xticks=[10^3, 10^4, 10^5])
plot!(xlabel="Number of facets")
plot!(ylabel="Calculation time [sec]")
plot!(xaxis=:log, yaxis=:log)

In [ ]:
# using FileIO, MeshIO
# mesh = load(shapepath)

In [ ]:
# using VTKView

In [ ]:
# using VTKDataTypes
# using VTKDataIO

In [ ]:
using VTKDataTypes
using WriteVTK
using PyCall
using LightXML
# using Iterators

@pyimport vtk.util.numpy_support as vtkns
@pyimport vtk as vtk
@pyimport numpy as np

In [ ]:
# include("/Users/masanorikanamaru/Documents/GitHub/VTKDataIO.jl/src/vtkreaders.jl")

In [ ]:
# read_vtk(vtkname)

In [ ]:
# read_static_vtk